In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
def getPureSample(raw_data,start,end,sr=22050):

    max_ind = len(raw_data)
    start_ind = min(int(start * sr), max_ind)
    end_ind = min(int(end * sr), max_ind)
    return raw_data[start_ind: end_ind]

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Classification-of-Lung-Sound/csv_data/data.csv')

In [5]:
print(data.start,data.end)

0        0.036
1        0.579
2        2.450
3        3.893
4        5.793
         ...  
6893    11.721
6894    13.693
6895    15.536
6896    17.493
6897    19.436
Name: start, Length: 6898, dtype: float64 0        0.579
1        2.450
2        3.893
3        5.793
4        7.521
         ...  
6893    13.693
6894    15.536
6895    17.493
6896    19.436
6897    19.979
Name: end, Length: 6898, dtype: float64


In [6]:
os.makedirs('processed_audio_files_test', exist_ok=True)

In [7]:
for index,row in data.iterrows():
    print("Index ->",index)
    print("Data->\n",row)
    break

Index -> 0
Data->
 start                        0.036
end                          0.579
crackles                         0
weezels                          0
pid                            101
mode                            sc
filename    101_1b1_Al_sc_Meditron
disease                       URTI
Name: 0, dtype: object


In [8]:
import os
import librosa as lb
import soundfile as sf

path = '/content/drive/MyDrive/Classification-of-Lung-Sound/upload_files_here_for_test/'  # Set the correct path
save_path = '/content/drive/MyDrive/Classification-of-Lung-Sound/processed_audio_files_test/'
maxLen = 6  # seconds

filename = 'test1'
start, end = row['start'], row['end']


audio_file_loc = os.path.join(path, filename + '.wav')

new_filename = filename + '_processed.wav'
save_file_loc = os.path.join(save_path, new_filename)

try:
    audioArr, sampleRate = lb.load(audio_file_loc)
    pureSample = getPureSample(audioArr, start, end, sampleRate)
    reqLen = 6 * sampleRate
    padded_data = lb.util.pad_center(pureSample, size=reqLen, axis=0)
    sf.write(file=save_file_loc, data=padded_data, samplerate=sampleRate)
    print(f"File processed successfully and saved at: {save_file_loc}")
except Exception as e:
    print(f"Error processing file {audio_file_loc}: {e}")


File processed successfully and saved at: /content/drive/MyDrive/Classification-of-Lung-Sound/processed_audio_files_test/test1_processed.wav


In [9]:
import librosa as lb
from keras.models import load_model
model=load_model('/content/drive/MyDrive/Classification-of-Lung-Sound/lungs-model-mfcc.h5')
def getFeatures(path):
    soundArr,sample_rate=lb.load(path)
    mfcc=lb.feature.mfcc(y=soundArr,sr=sample_rate)
    cstft=lb.feature.chroma_stft(y=soundArr,sr=sample_rate)
    mSpec=lb.feature.melspectrogram(y=soundArr,sr=sample_rate)

    return mfcc,cstft,mSpec


In [10]:
mfcc, cstft, mSpec = getFeatures("/content/drive/MyDrive/Classification-of-Lung-Sound/processed_audio_files_test/"+new_filename)

In [11]:
mfcc_padded = np.zeros((1, 20, 259))
mfcc_padded[0, :mfcc.shape[0], :mfcc.shape[1]] = mfcc

cstft_padded = np.zeros((1, 12, 259))
cstft_padded[0, :cstft.shape[0], :cstft.shape[1]] = cstft

mSpec_padded = np.zeros((1, 128, 259))
mSpec_padded[0, :mSpec.shape[0], :mSpec.shape[1]] = mSpec

In [12]:
mfcc_val = np.array(mfcc_padded)
print(mfcc_val)

[[[-638.89202881 -638.89202881 -638.89202881 ... -638.89202881
   -638.89202881 -638.89202881]
  [   0.            0.            0.         ...    0.
      0.            0.        ]
  [   0.            0.            0.         ...    0.
      0.            0.        ]
  ...
  [   0.            0.            0.         ...    0.
      0.            0.        ]
  [   0.            0.            0.         ...    0.
      0.            0.        ]
  [   0.            0.            0.         ...    0.
      0.            0.        ]]]


In [13]:
cstft_val = np.array(cstft_padded)
print(cstft_val)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [14]:
mSpec_val = np.array(mSpec_padded)
print(mSpec_val)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [15]:
result=model.predict({"mfcc":mfcc_val,"croma":cstft_val,"mspec":mSpec_val}, batch_size=32)
print(result)

1/1 [==============================] - 1s 550ms/step
[[4.6052717e-09 3.5996879e-06 1.2244448e-08 9.9934012e-01 3.2070974e-07
  3.0321266e-11 6.5599475e-04 1.4025073e-09]]


In [16]:
int_to_label = {0: 'URTI', 1: 'Healthy', 2: 'Asthma', 3: 'COPD', 4: 'LRTI', 5: 'Bronchiectasis', 6: 'Pneumonia', 7: 'Bronchiolitis'}

In [17]:
# Extracting classes from predictions using argmax
y_pred = np.argmax(result, axis=1)

print("Prediction test return:", y_pred[0], "-", int_to_label[y_pred[0]])

Prediction test return: 3 - COPD
